In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import warnings

In [2]:
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')

In [3]:
sales_ = pd.read_csv('Online Retail dataset.csv') ##,encoding='unicode_escape'

In [4]:
sales_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
sales_.dropna(subset='CustomerID',axis=0,inplace=True) 
sales_['CustomerID'] = sales_['CustomerID'].astype('int') 
sales_['InvoiceDate'] = sales_['InvoiceDate'].astype('datetime64[ns]')
sales_.drop_duplicates()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [6]:
sales_ = sales_[((sales_['Quantity']>0)&(sales_['UnitPrice']>0))&(sales_['Country']!='Unspecified')& (sales_['Country']!='European Community')] ## Removing  quantities and unitprices those are less than 0 as we shouldnt get misleading results

In [7]:
sales_['Monetary'] = sales_['UnitPrice']*sales_['Quantity']

In [8]:
last_purchase_day = sales_['InvoiceDate'].max()

In [9]:
rfm_table = pd.DataFrame(data = sales_.groupby(by='CustomerID').agg({'InvoiceDate': lambda x: (last_purchase_day - x.max()).days,
                                'InvoiceNo': lambda y: y.nunique(),
                                'Monetary': lambda z: z.sum()}).reset_index())

rfm_table.rename(columns={'InvoiceDate':'Recency','InvoiceNo':'Frequency'},inplace=True)

In [10]:
quantiles = rfm_table[['Recency','Frequency','Monetary']].quantile([0.25,0.5,0.75])

In [11]:
quantiles

,Recency,Frequency,Monetary
0.25,17.0,1.0,307.09
0.50,50.0,2.0,674.52
0.75,141.0,5.0,1662.28


In [12]:
rfm_table 
## recency < 
## frequency & monetary value >

,CustomerID,Recency,Frequency,Monetary
0,12346,325,1,77183.60
1,12347,1,7,4310.00
2,12348,74,4,1797.24
3,12349,18,1,1757.55
4,12350,309,1,334.40
...,...,...,...,...
4328,18280,277,1,180.60
4329,18281,180,1,80.82
4330,18282,7,2,178.05
4331,18283,3,16,2094.88


In [13]:
def FM_Score(value,q_tab,col): ##Frequency and monetary value 
    if value <= q_tab[col][0.25]: 
        return 4 
    elif value <=q_tab[col][0.5]: 
        return 3
    elif value <=q_tab[col][0.75]: 
        return 2
    else: 
        return 1

def R_Score(value,q_tab,col): ##Recency Score
    if value <= q_tab[col][0.25]: 
        return 1
    elif value <=q_tab[col][0.5]: 
        return 2
    elif value <=q_tab[col][0.75]: 
        return 3 
    else: 
        return 4

In [14]:
rfm_table['R_Score']  = rfm_table['Recency'].apply(R_Score,args=(quantiles,'Recency'))
rfm_table['F_Score']  = rfm_table['Frequency'].apply(FM_Score,args=(quantiles,'Frequency'))
rfm_table['M_Score']  = rfm_table['Monetary'].apply(FM_Score,args=(quantiles,'Monetary'))

In [15]:
def change_dtype(r_Score,f_Score,m_Score):
    return str(r_Score) + str(f_Score) + str(m_Score)

rfm_table['RFM_Score'] = np.vectorize(change_dtype)(rfm_table['R_Score'],rfm_table['F_Score'],rfm_table['M_Score'])

In [16]:
rfm_table

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,12346,325,1,77183.60,4,4,1,441
1,12347,1,7,4310.00,1,1,1,111
2,12348,74,4,1797.24,3,2,1,321
3,12349,18,1,1757.55,2,4,1,241
4,12350,309,1,334.40,4,4,3,443
...,...,...,...,...,...,...,...,...
4328,18280,277,1,180.60,4,4,4,444
4329,18281,180,1,80.82,4,4,4,444
4330,18282,7,2,178.05,1,3,4,134
4331,18283,3,16,2094.88,1,1,1,111


In [17]:
def segment_customer(sales_):
    if sales_['R_Score'] ==1 and sales_['F_Score']==1 and sales_['M_Score'] ==1: 
        return 'Best Customers'
    elif sales_['R_Score'] ==3 and sales_['F_Score']==1 and sales_['M_Score'] ==1: 
        return 'Almost Lost'
    elif sales_['R_Score'] ==4 and sales_['F_Score']==1 and sales_['M_Score'] ==1: 
        return 'Lost Customers'
    elif sales_['R_Score'] ==4 and sales_['F_Score']==4 and sales_['M_Score'] ==4: 
        return 'Lost Cheap Customers'
    elif sales_['F_Score'] ==1: 
        return 'Loyal Customers'
    elif sales_['M_Score']==1: 
        return 'Big Spenders'
    else: 
        return 'Others'

In [18]:
rfm_table['Segmented'] = rfm_table.apply(segment_customer,axis=1)

In [19]:
rfm_table

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segmented
0,12346,325,1,77183.60,4,4,1,441,Big Spenders
1,12347,1,7,4310.00,1,1,1,111,Best Customers
2,12348,74,4,1797.24,3,2,1,321,Big Spenders
3,12349,18,1,1757.55,2,4,1,241,Big Spenders
4,12350,309,1,334.40,4,4,3,443,Others
...,...,...,...,...,...,...,...,...,...
4328,18280,277,1,180.60,4,4,4,444,Lost Cheap Customers
4329,18281,180,1,80.82,4,4,4,444,Lost Cheap Customers
4330,18282,7,2,178.05,1,3,4,134,Others
4331,18283,3,16,2094.88,1,1,1,111,Best Customers


In [20]:
rfm_table = rfm_table.merge(sales_[['CustomerID','Country']],how='inner',on='CustomerID').drop_duplicates()

In [21]:
cnt_customers = rfm_table.groupby(by=['Country','Segmented'])['CustomerID'].count().reset_index()
cnt_customers

,Country,Segmented,CustomerID
0,Australia,Best Customers,1
1,Australia,Big Spenders,1
2,Australia,Loyal Customers,2
3,Australia,Others,5
4,Austria,Big Spenders,3
...,...,...,...
94,United Kingdom,Big Spenders,283
95,United Kingdom,Lost Cheap Customers,414
96,United Kingdom,Lost Customers,10
97,United Kingdom,Loyal Customers,298


In [63]:
pivot_ = pd.pivot_table(
    data=cnt_customers,columns='Country',index='Segmented',values='CustomerID',fill_value=0,margins=True,margins_name='Grand Total',aggfunc=np.sum
).sort_values('Grand Total', axis=1, ascending=False)

In [64]:
pivot_ = pivot_.loc[pivot_.index!='Grand Total',pivot_.columns!='Grand Total'].style.background_gradient(cmap='PuBu')

In [65]:
pivot_

Country,United Kingdom,Germany,France,Spain,Belgium,Switzerland,Portugal,Italy,Finland,Austria,Norway,Netherlands,Australia,Channel Islands,Denmark,Sweden,Cyprus,Japan,Poland,Greece,Canada,USA,EIRE,Israel,Malta,Bahrain,United Arab Emirates,Brazil,Lebanon,Singapore,Lithuania,RSA,Iceland,Czech Republic,Saudi Arabia
Segmented,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Almost Lost,63,0,2,1,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Best Customers,420,13,11,1,3,0,3,1,2,0,1,1,1,0,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0
Big Spenders,283,16,9,5,7,7,4,3,2,3,5,0,1,3,3,0,2,3,0,1,1,0,1,2,1,0,0,0,1,0,0,0,0,0,0
Lost Cheap Customers,414,5,11,2,3,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
Lost Customers,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Loyal Customers,298,9,10,3,3,0,0,1,1,0,1,1,2,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Others,2432,51,44,18,9,12,11,9,7,7,3,6,5,5,6,5,3,4,5,3,2,4,0,1,1,1,2,1,0,0,1,1,0,1,0
